In [1]:
import string
import numpy as np

In [2]:
abc = ' .,' + string.ascii_lowercase
len_abc = len(abc)

itoc = {i:c for c,i in zip(abc, range(len_abc))}
ctoi = {c:i for c,i in zip(abc, range(len_abc))}

def encode(text: str) -> list[int]:
    return [ctoi[c] for c in text]

def decode(idxs: list[int]) -> str:
    return ''.join([itoc[i] for i in idxs])

In [3]:
def onehot(n: int, i: int) -> np.ndarray:
    vector = np.zeros(n)
    vector[i] = 1.0
    return vector

def str2seq(text: str) -> np.ndarray:
    return np.array([onehot(len_abc, i) for i in encode(text)])

def seq2str(seq: np.ndarray) -> str:
    return decode([int(np.argmax(v)) for v in seq])

In [4]:
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

### Define model

In [5]:
class RNN:
    def __init__(self, n_in: int, n_h: int, n_out: int) -> None:
        self.W_ih: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.W_hh: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.b_h: np.ndarray = np.zeros(n_h)

        self.W_ho: np.ndarray = np.random.uniform(-1, 1, (n_out, n_h))
        self.b_ho: np.ndarray = np.zeros(n_out)
    
    @property
    def parameters(self) -> tuple[np.ndarray]:
        return (self.W_ih, self.W_hh, self.b_h, self.W_ho, self.b_ho)
    
    def many_to_one(self, sequence: np.ndarray) -> tuple[np.ndarray]:
        context: np.ndarray = np.zeros(len(self.W_hh))
        
        for x in sequence:
            context = np.tanh(self.W_ih @ x + self.W_hh @ context + self.b_h)
        output = softmax(self.W_ho @ context + self.b_ho)
        
        return (output, context)

    def one_to_many(self, x: np.ndarray, t: int) -> tuple[np.ndarray]:
        outputs: np.ndarray = np.zeros((t, len(self.W_ho)))
        context = np.tanh(self.W_ih @ x + self.b_h)

        for i in range(t):
            outputs[i] = softmax(self.W_ho @ context + self.b_ho)
            context = np.tanh(self.W_ih @ outputs[i] + self.W_hh @ context + self.b_h)

        return (outputs, context)

    def many_to_many(self, sequence: np.ndarray) -> tuple[np.ndarray]:
        outputs: np.ndarray = np.zeros((len(sequence), len(self.W_ho)))
        context: np.ndarray = np.zeros(len(self.W_hh))

        for i,x in enumerate(sequence):
            context = np.tanh(self.W_ih @ x + self.W_hh @ context + self.b_h)
            outputs[i] = softmax(self.W_ho @ context + self.b_ho)
        
        return (outputs, context)
    
    def seq_to_seq(self, sequence: np.ndarray, t: int) -> tuple[np.ndarray]:
        outputs: np.ndarray = np.zeros((t, len(self.W_ho)))
        context: np.ndarray = np.zeros(len(self.W_hh))

        for i,x in enumerate(sequence):
            context = np.tanh(self.W_ih @ x + self.W_hh @ context + self.b_h)
        
        for i in range(t):
            outputs[i] = softmax(self.W_ho @ context + self.b_ho)
            context = np.tanh(self.W_ih @ outputs[i-1] + self.W_hh @ context + self.b_h)
        
        return (outputs, context)


### Create and run model

In [11]:
n_in = len_abc
n_h = 64
n_out = len_abc

model = RNN(n_in, n_h, n_out)

In [12]:
text = 'the quick brown fox jumps over the lazy dog'
seq = str2seq(text)

outputs, context = model.many_to_one(seq)
print(abc[np.argmax(outputs)])

u


In [13]:
text = 't'
seq = str2seq(text)

outputs, context = model.one_to_many(seq[0], 128)
print(seq2str(outputs))

wykejio,ryreyz.mzuuhsk,duiicuafauyddndkdmlu.ii,okdhtvdqaeqrbiiuapnbmhjqrry.xiiaykpkdttqqsevu .mjnub,urtppfhhwabq.uiwvrmfz bdpedz


In [14]:
text = 'the quick brown fox jumps over the lazy dog'
seq = str2seq(text)

outputs, context = model.many_to_many(seq)
print(seq2str(outputs))

wgvq uownmkzqhfdqiruskwqebdijnuziue,yfpupxu


In [15]:
text = 'the quick brown fox jumps over the lazy dog'
seq = str2seq(text)

outputs, context = model.seq_to_seq(seq, 128)
print(seq2str(outputs))

ushuybpta caah gbtld,adfchttqqgu.mia.dpmhw v lcbxfzfhihpsapqab c,r.midpp.a.huhgpqcvwb ukmy,tdiwucku.y,.uorim.hh.hbtfe.c xfeidqwy
